In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
df_train = pd.read_csv('../input/digit-recognizer/train.csv')
df_test = pd.read_csv('../input/digit-recognizer/test.csv')

In [4]:
X_train = df_train.loc[:,df_train.columns != 'label'].to_numpy().reshape(-1,28,28)
y_train = df_train['label'].to_numpy()
X_test = df_test.to_numpy().reshape(-1,28,28)

In [5]:
X_train = (X_train / 255.)[...,np.newaxis]
X_test = (X_test / 255.)[...,np.newaxis]

In [6]:
X_train, X_valid = X_train[:-5000], X_train[-5000:]
y_train, y_valid = y_train[:-5000], y_train[-5000:]

In [7]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=3, padding="same", activation="relu"),
    keras.layers.Conv2D(64, kernel_size=3, padding="same", activation="relu"),
    keras.layers.MaxPool2D(),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])

2022-06-04 11:56:23.619534: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [8]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

2022-06-04 11:56:24.104278: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
1157/1157 [==============================] - 59s 50ms/step - loss: 0.2283 - accuracy: 0.9315 - val_loss: 0.0680 - val_accuracy: 0.9782
Epoch 2/10
1157/1157 [==============================] - 58s 50ms/step - loss: 0.0900 - accuracy: 0.9723 - val_loss: 0.0468 - val_accuracy: 0.9850
Epoch 3/10
1157/1157 [==============================] - 57s 49ms/step - loss: 0.0658 - accuracy: 0.9798 - val_loss: 0.0457 - val_accuracy: 0.9856
Epoch 4/10
1157/1157 [==============================] - 58s 50ms/step - loss: 0.0534 - accuracy: 0.9825 - val_loss: 0.0424 - val_accuracy: 0.9858
Epoch 5/10
1157/1157 [==============================] - 57s 50ms/step - loss: 0.0480 - accuracy: 0.9845 - val_loss: 0.0457 - val_accuracy: 0.9862
Epoch 6/10
1157/1157 [==============================] - 57s 50ms/step - loss: 0.0403 - accuracy: 0.9870 - val_loss: 0.0443 - val_accuracy: 0.9876
Epoch 7/10
1157/1157 [==============================] - 57s 50ms/step - loss: 0.0370 - accuracy: 0.9884 - val_loss: 0.0503 -

In [9]:
submission = model.predict(X_test)

In [10]:
submission = submission.argmax(1)

In [11]:
submission.shape

(28000,)

In [12]:
df_sample = pd.read_csv('../input/digit-recognizer/sample_submission.csv')

In [13]:
df_sample['Label'] = submission

In [14]:
df_sample.to_csv('sumission.csv')